# 目的

- nb001で売り買いできることがわかったから戦略を考える

- bitfrayerの取引データをどこかから探してくる！（ろうそく足データ）

https://blog.shoby.jp/entry/2017/05/24/093733#f-e6c220ff

- lightning fxでmmbotを運用する

In [1]:
import os 
import sys 
import time
import numpy as np
import pandas as pd
import talib

# 仮想通貨関連
import ccxt
import pybitflyer

# open high low close V

---

# richmanbtcさんの特徴量作成メソッド

In [2]:
def calc_features(df):
    open = df['op']
    high = df['hi']
    low = df['lo']
    close = df['cl']
    volume = df['volume']
    
    orig_columns = df.columns

    hilo = (df['hi'] + df['lo']) / 2
    df['BBANDS_upperband'], df['BBANDS_middleband'], df['BBANDS_lowerband'] = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    df['BBANDS_upperband'] -= hilo
    df['BBANDS_middleband'] -= hilo
    df['BBANDS_lowerband'] -= hilo
    df['DEMA'] = talib.DEMA(close, timeperiod=30) - hilo
    df['EMA'] = talib.EMA(close, timeperiod=30) - hilo
    df['HT_TRENDLINE'] = talib.HT_TRENDLINE(close) - hilo
    df['KAMA'] = talib.KAMA(close, timeperiod=30) - hilo
    df['MA'] = talib.MA(close, timeperiod=30, matype=0) - hilo
    df['MIDPOINT'] = talib.MIDPOINT(close, timeperiod=14) - hilo
    df['SMA'] = talib.SMA(close, timeperiod=30) - hilo
    df['T3'] = talib.T3(close, timeperiod=5, vfactor=0) - hilo
    df['TEMA'] = talib.TEMA(close, timeperiod=30) - hilo
    df['TRIMA'] = talib.TRIMA(close, timeperiod=30) - hilo
    df['WMA'] = talib.WMA(close, timeperiod=30) - hilo

    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['ADXR'] = talib.ADXR(high, low, close, timeperiod=14)
    df['APO'] = talib.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df['AROON_aroondown'], df['AROON_aroonup'] = talib.AROON(high, low, timeperiod=14)
    df['AROONOSC'] = talib.AROONOSC(high, low, timeperiod=14)
    df['BOP'] = talib.BOP(open, high, low, close)
    df['CCI'] = talib.CCI(high, low, close, timeperiod=14)
    df['DX'] = talib.DX(high, low, close, timeperiod=14)
    df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    # skip MACDEXT MACDFIX たぶん同じなので
    df['MFI'] = talib.MFI(high, low, close, volume, timeperiod=14)
    df['MINUS_DI'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    df['MINUS_DM'] = talib.MINUS_DM(high, low, timeperiod=14)
    df['MOM'] = talib.MOM(close, timeperiod=10)
    df['PLUS_DI'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['PLUS_DM'] = talib.PLUS_DM(high, low, timeperiod=14)
    df['RSI'] = talib.RSI(close, timeperiod=14)
    df['STOCH_slowk'], df['STOCH_slowd'] = talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df['STOCHF_fastk'], df['STOCHF_fastd'] = talib.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = talib.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['TRIX'] = talib.TRIX(close, timeperiod=30)
    df['ULTOSC'] = talib.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    df['WILLR'] = talib.WILLR(high, low, close, timeperiod=14)

    df['AD'] = talib.AD(high, low, close, volume)
    df['ADOSC'] = talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
    df['OBV'] = talib.OBV(close, volume)

    df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
    df['NATR'] = talib.NATR(high, low, close, timeperiod=14)
    df['TRANGE'] = talib.TRANGE(high, low, close)

    df['HT_DCPERIOD'] = talib.HT_DCPERIOD(close)
    df['HT_DCPHASE'] = talib.HT_DCPHASE(close)
    df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(close)
    df['HT_SINE_sine'], df['HT_SINE_leadsine'] = talib.HT_SINE(close)
    df['HT_TRENDMODE'] = talib.HT_TRENDMODE(close)

    df['BETA'] = talib.BETA(high, low, timeperiod=5)
    df['CORREL'] = talib.CORREL(high, low, timeperiod=30)
    df['LINEARREG'] = talib.LINEARREG(close, timeperiod=14) - close
    df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(close, timeperiod=14)
    df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(close, timeperiod=14) - close
    df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(close, timeperiod=14)
    df['STDDEV'] = talib.STDDEV(close, timeperiod=5, nbdev=1)

    return df

---

# Main 

主処理

In [3]:
df_btc = pd.read_csv("../data/nb002/btc_jpy_15min.csv")
df_btc.columns = ["time", "op", "hi", "lo", "cl", "volume"]

In [4]:
df_btc

,time,op,hi,lo,cl,volume
0,2021-10-27 13:15:00,6720546,6738258,6710000,6735988,20.589480
1,2021-10-27 13:30:00,6738258,6739563,6708000,6710000,14.918648
2,2021-10-27 13:45:00,6711247,6722993,6700000,6722993,22.997577
3,2021-10-27 14:00:00,6720612,6736192,6704518,6710000,17.630379
4,2021-10-27 14:15:00,6713667,6723157,6704320,6704320,16.902241
...,...,...,...,...,...,...
5994,2021-12-28 23:45:00,5475000,5484560,5468904,5480311,16.271515
5995,2021-12-29 00:00:00,5478197,5480487,5454215,5456831,17.486344
5996,2021-12-29 00:15:00,5455923,5488466,5447625,5488285,38.533803
5997,2021-12-29 00:30:00,5487234,5490928,5477848,5481000,19.962052


In [4]:
# 12月データをテストとして切り出し
df_btc_train = df_btc[pd.to_datetime(df_btc["time"]) < pd.to_datetime("2021-12-01")]
df_btc_test = df_btc[pd.to_datetime(df_btc["time"]) >= pd.to_datetime("2021-12-01")]

In [5]:
df_btc_train, df_btc_test

(                     time       op       hi       lo       cl     volume
 0     2021-10-27 13:15:00  6720546  6738258  6710000  6735988  20.589480
 1     2021-10-27 13:30:00  6738258  6739563  6708000  6710000  14.918648
 2     2021-10-27 13:45:00  6711247  6722993  6700000  6722993  22.997577
 3     2021-10-27 14:00:00  6720612  6736192  6704518  6710000  17.630379
 4     2021-10-27 14:15:00  6713667  6723157  6704320  6704320  16.902241
 ...                   ...      ...      ...      ...      ...        ...
 3302  2021-11-30 22:45:00  6504166  6506439  6485316  6500000   6.466320
 3303  2021-11-30 23:00:00  6496913  6500930  6479772  6491062   5.330892
 3304  2021-11-30 23:15:00  6488943  6503746  6462535  6470978  23.821248
 3305  2021-11-30 23:30:00  6475942  6501111  6472106  6493391  16.240694
 3306  2021-11-30 23:45:00  6493392  6495810  6466255  6479057  21.825394
 
 [3307 rows x 6 columns],
                      time       op       hi       lo       cl     volume
 3307  202

---

In [6]:
!pip install pybitflyer
import pybitflyer
pybitflyer.API().board(product_code='FX_BTC_JPY')

/bin/bash: pip: command not found


{'mid_price': 5493149.0,
 'bids': [{'price': 5492516.0, 'size': 0.01},
  {'price': 5492512.0, 'size': 0.01},
  {'price': 5491363.0, 'size': 0.17},
  {'price': 5490985.0, 'size': 0.06191996},
  {'price': 5490884.0, 'size': 0.17},
  {'price': 5490749.0, 'size': 0.01},
  {'price': 5490718.0, 'size': 0.01},
  {'price': 5490391.0, 'size': 0.0128},
  {'price': 5490364.0, 'size': 0.17},
  {'price': 5490361.0, 'size': 0.17},
  {'price': 5490280.0, 'size': 0.02},
  {'price': 5490259.0, 'size': 0.017393},
  {'price': 5489752.0, 'size': 0.01},
  {'price': 5489602.0, 'size': 0.02},
  {'price': 5489601.0, 'size': 0.05533831},
  {'price': 5489393.0, 'size': 0.01},
  {'price': 5489160.0, 'size': 0.01768394},
  {'price': 5489146.0, 'size': 0.01767413},
  {'price': 5489142.0, 'size': 0.0176736},
  {'price': 5489133.0, 'size': 0.01767756},
  {'price': 5489111.0, 'size': 0.1},
  {'price': 5489022.0, 'size': 0.01},
  {'price': 5489004.0, 'size': 0.17},
  {'price': 5488974.0, 'size': 0.01},
  {'price': 548

# 戦略 - ストラテジーを考える

## シンプル

- 上げトレンド→買い

- 下げトレンド→売り

In [8]:
# ここのハードコーディング危険
api_key = '82BLyXP8xgK2JLjsKFMajV'
api_secret = 'uYu8qv+y9F7EBjzhfa2kitidpVfdyH4/riCopxQho+k='

In [26]:
api = pybitflyer.API(api_key=api_key, api_secret=api_secret)

In [106]:
buy_btc = api.sendchildorder(product_code="FX_BTC_JPY",
                             child_order_type="LIMIT",
                             side="BUY",
                             size=0.01,
                             price=5535191,
                             minute_to_expire=10000,
                             time_in_force="GTC"
                             )

In [104]:
base_price = api.ticker(product_code="FX_BTC_JPY")["ltp"]
sell_price = base_price - (base_price*0.003)
base_price

5546829.0

In [105]:
api.board(product_code="FX_BTC_JPY")['bids']

[{'price': 5547108.0, 'size': 0.01},
 {'price': 5546427.0, 'size': 0.02},
 {'price': 5546264.0, 'size': 0.01425864},
 {'price': 5546247.0, 'size': 0.14},
 {'price': 5546153.0, 'size': 0.03},
 {'price': 5546049.0, 'size': 0.00262214},
 {'price': 5545997.0, 'size': 0.18},
 {'price': 5545891.0, 'size': 0.04765852},
 {'price': 5545888.0, 'size': 0.01},
 {'price': 5545885.0, 'size': 0.00752743},
 {'price': 5545659.0, 'size': 0.12781098},
 {'price': 5545503.0, 'size': 0.02382846},
 {'price': 5545497.0, 'size': 0.1},
 {'price': 5545477.0, 'size': 0.18},
 {'price': 5545476.0, 'size': 0.18},
 {'price': 5545294.0, 'size': 0.03391848},
 {'price': 5545280.0, 'size': 0.02},
 {'price': 5545194.0, 'size': 0.014},
 {'price': 5545190.0, 'size': 0.02},
 {'price': 5545178.0, 'size': 0.01},
 {'price': 5545034.0, 'size': 0.1472},
 {'price': 5545033.0, 'size': 0.02},
 {'price': 5544658.0, 'size': 0.04983149},
 {'price': 5544651.0, 'size': 0.01606814},
 {'price': 5544268.0, 'size': 0.02},
 {'price': 5544095.

In [109]:
api.board(product_code="FX_BTC_JPY")['asks']

[{'price': 5549915.0, 'size': 0.03},
 {'price': 5549985.0, 'size': 0.011},
 {'price': 5550066.0, 'size': 0.087},
 {'price': 5550770.0, 'size': 0.01},
 {'price': 5550942.0, 'size': 0.01},
 {'price': 5550983.0, 'size': 0.2021},
 {'price': 5551014.0, 'size': 0.01},
 {'price': 5551227.0, 'size': 0.01},
 {'price': 5551306.0, 'size': 0.01},
 {'price': 5551401.0, 'size': 0.02},
 {'price': 5551472.0, 'size': 0.18},
 {'price': 5551508.0, 'size': 0.01574652},
 {'price': 5551517.0, 'size': 0.01},
 {'price': 5551732.0, 'size': 0.04},
 {'price': 5551908.0, 'size': 0.01},
 {'price': 5551929.0, 'size': 0.02},
 {'price': 5551984.0, 'size': 0.02380688},
 {'price': 5551985.0, 'size': 0.02380523},
 {'price': 5551995.0, 'size': 0.025},
 {'price': 5551996.0, 'size': 0.18},
 {'price': 5552015.0, 'size': 0.02},
 {'price': 5552081.0, 'size': 0.01},
 {'price': 5552125.0, 'size': 0.02},
 {'price': 5552264.0, 'size': 0.01},
 {'price': 5552313.0, 'size': 0.03},
 {'price': 5552316.0, 'size': 0.1},
 {'price': 55524

In [116]:
sell_btc = api.sendchildorder(product_code="FX_BTC_JPY",
                             child_order_type="LIMIT",
                             side="SELL",
                             size=0.01,
                             price=5518086,
                             minute_to_expire=10000,
                             time_in_force="GTC"
                             )

In [117]:
api.board(product_code="FX_BTC_JPY")['bids']

[{'price': 5510027.0, 'size': 0.02},
 {'price': 5510025.0, 'size': 0.18},
 {'price': 5510024.0, 'size': 0.01},
 {'price': 5509408.0, 'size': 0.01},
 {'price': 5509275.0, 'size': 0.01},
 {'price': 5509004.0, 'size': 0.01},
 {'price': 5508969.0, 'size': 0.06216488},
 {'price': 5508965.0, 'size': 0.07259668},
 {'price': 5508954.0, 'size': 0.15},
 {'price': 5508953.0, 'size': 0.02250856},
 {'price': 5508952.0, 'size': 0.02284782},
 {'price': 5508825.0, 'size': 0.18},
 {'price': 5508506.0, 'size': 0.01},
 {'price': 5508203.0, 'size': 0.01},
 {'price': 5507965.0, 'size': 0.02045159},
 {'price': 5507627.0, 'size': 0.01},
 {'price': 5507247.0, 'size': 0.04},
 {'price': 5507238.0, 'size': 0.02},
 {'price': 5507237.0, 'size': 0.01},
 {'price': 5507233.0, 'size': 0.01},
 {'price': 5507089.0, 'size': 0.01},
 {'price': 5507036.0, 'size': 0.18},
 {'price': 5506597.0, 'size': 0.18},
 {'price': 5506481.0, 'size': 0.01},
 {'price': 5506417.0, 'size': 0.02},
 {'price': 5506031.0, 'size': 0.1},
 {'price'

--> lightning fxもできることがわかった！